In [ ]:
import json
import sys
import hvplot.pandas
import pandas as pd
from utils import load_config, fetch_api_data, write_to_csv

In [ ]:
#https://www.spc.noaa.gov/wcm/#data

In [ ]:
#https://www.ncdc.noaa.gov/swdiws
# 'nx3tvs'       - (Point)   NEXRAD Level-3 Tornado Vortex Signatures
# 'nx3meso'      - (Point)   NEXRAD Level-3 Mesocyclone Signatures
# 'nx3hail'      - (Point)   NEXRAD Level-3 Hail Signatures
# 'nx3structure' - (Point)   NEXRAD Level-3 Storm Cell Structure Information
# 'warn'         - (Polygon) Severe Thunderstorm, Tornado, Flash Flood and Special Marine warnings
datasets = ["nx3tvs"]
outputFormat = "json"
daterange = "20240701:20240731"  # "periodOfRecord"

for dataset in datasets:
    base_url = f"https://www.ncdc.noaa.gov/swdiws/{outputFormat}/{dataset}/{daterange}"
    filename = f"swdiws_{dataset}.csv"

    data = fetch_api_data(base_url)
    #print(json.dumps(data, indent=2))
    #print(data)

    if data and "result" in data:
        write_to_csv(data["result"], filename, "w")
    else:
        print(f"No data found or invalid response format for dataset: {dataset}")

In [ ]:
# 'nx3tvs'       - (Point)   NEXRAD Level-3 Tornado Vortex Signatures
# 'nx3meso'      - (Point)   NEXRAD Level-3 Mesocyclone Signatures
# 'nx3hail'      - (Point)   NEXRAD Level-3 Hail Signatures
# 'nx3structure' - (Point)   NEXRAD Level-3 Storm Cell Structure Information
# 'warn'         - (Polygon) Severe Thunderstorm, Tornado, Flash Flood and Special Marine warnings
datasets = ["nx3tvs"]
outputFormat = "geojson"
daterange = "20240701:20240731"  # "periodOfRecord"
numResults = 2500

# Initialize an empty list to store merged data
merged_data_list = []

for dataset in datasets:
    base_url = f"https://www.ncdc.noaa.gov/swdiws/{outputFormat}/{dataset}/{daterange}/{numResults}"
    filename = f"swdiws_{dataset}_{outputFormat}.csv"

    data = fetch_api_data(base_url)
    # Data is nested, retrieve "features" dictionary
    rows = data["features"]
    
    # Iterate over each record in rows as that is nested as well
    for record in rows:
       # Merge the 'properties' and 'geometry' dictionaries
        merged_data = {**record["properties"], **record["geometry"]}
        # Append the merged data to the list
        merged_data_list.append(merged_data)

    # Convert to DataFrame
    merged_df = pd.DataFrame(merged_data_list)

# Split the coordinates column into latitude and longitude
merged_df[['longitude', 'latitude']] = pd.DataFrame(merged_df['coordinates'].tolist(), index=merged_df.index)
# Drop the original coordinates column
merged_df = merged_df.drop(columns=['coordinates'])
merged_df.head(25)

    # if data and "features" in data:
    #     write_to_csv(data["features"], filename, "w")
    # else:
    #     print(f"No data found or invalid response format for dataset: {dataset}")


In [ ]:
# Convert 'MXDV' to numeric, forcing errors to NaN if conversion fails
merged_df['MXDV'] = pd.to_numeric(merged_df['MXDV'], errors='coerce')
# Ensure 'WSR_ID' is treated as a string
merged_df['WSR_ID'] = merged_df['WSR_ID'].astype(str)
merged_df.to_csv('swdiws_nx3tvs_geojson.csv', index=False)
print(merged_df.dtypes)

In [ ]:
# https://www.fema.gov/about/openfema/data-sets
base_url = f"https://www.fema.gov/api/open/v2/"

params = {"$count": "true",
          "$filter": "incidentType eq 'Tornado'"}

endpoint = "DisasterDeclarationsSummaries"
filename = f"{endpoint}.csv"
endpoint_url = f"{base_url}{endpoint}"

data = fetch_api_data(endpoint_url, params)
#print(json.dumps(data, indent=2))
tornado_summary_df = pd.DataFrame(data["DisasterDeclarationsSummaries"])
write_to_csv(data["DisasterDeclarationsSummaries"], filename, "w")

disaster_numbers = tornado_summary_df.disasterNumber.unique()
formatted_disaster_numbers = ', '.join(f'{num}' for num in disaster_numbers)
formatted_disaster_numbers

    #disasterNumber
#tornado_newer_df = tornado_df[tornado_df["declarationDate"] > "2019-12-31"]
#tornado_df.groupby(by="disasterNumber").count()


In [ ]:
params = {"$count": "true",
          "$filter": f"disasterNumber in ({formatted_disaster_numbers})"}

endpoint = "HousingAssistanceOwners"
filename = f"{endpoint}.csv"
endpoint_url = f"{base_url}{endpoint}"
data = fetch_api_data(endpoint_url, params)
#print(json.dumps(data, indent=2))
housing_assistance_df = pd.DataFrame(data["HousingAssistanceOwners"])
write_to_csv(data["HousingAssistanceOwners"], filename, "w")
housing_assistance_df

In [ ]:
%%capture --no-display
# Configure the map plot
tornadoes_plot = merged_df.hvplot.points(
    "longitude",
    "latitude",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="MXDV",
    color="WSR_ID"
)

# Display the map
tornadoes_plot

In [ ]:
### National Centers for Environmental Information ###

# Load config file
config = load_config('config.json')

# Get API key
weather_api_key = config.get('National Centers for Environmental Information', {}).get('key')

# Set base URL
base_url = f"https://www.ncei.noaa.gov/cdo-web/api/v2"

# Set endpoints
endpoints = ["datasets", "datacategories", "datatypes", "locationcategories", "locations", "stations", "data"]

# Check if key exists in config file 
if not weather_api_key:
    print("Weather API key not found in the configuration file.")
    sys.exit()
# Define headers
headers = {"token": weather_api_key}
# Define HTTPS call timeout (seconds)
timeout = 30

# Loop through each endpoint
for endpoint in endpoints:
    print(f"Running: {endpoint}")
    
    # Update params if the endpoint is 'data'
    if endpoint == "data":
        params = {"datasetid": "GHCND"}
        timeout = 60
    else:
        # Ensure 'datasetid' is not in params for other endpoints
        params = {"limit": 1000}

    # Define output filename
    filename = f"NCEI_{endpoint}.csv"
    
    # Set URL
    url_endpoint = f"{base_url}/{endpoint}"
    
    # Run a request using params and header dictionaries
    data = fetch_api_data(url_endpoint, params, headers, "GET", timeout)
    
    # Process and write the data to CSV
    if data and "results" in data:
        write_to_csv(data["results"], filename, "w")
    else:
        print(f"No data found or invalid response format for endpoint: {endpoint}")

In [ ]:
# API to convert coordinates to addresses
lat = "41.79445"
lng = "-72.89675"

params = {"format": "json",
          "lat": lat,
          "lon": lng}

headers = {"User-Agent": "MyCustomApp/1.0 (hef1125@hotmail.com)"}

endpoint = "reverse"

base_url = f"https://nominatim.openstreetmap.org/{endpoint}"


data = fetch_api_data(base_url, params, headers)
print(json.dumps(data, indent=2))